***GENERATED CODE FOR srimass PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
import pyspark
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
import statsmodels.api as sm


def driverSarima(df):
    df = df.set_index(df.columns[0])
    objSARIMA = sm.tsa.statespace.SARIMAX(
        df, order=[0, 1, 1], seasonal_order=[0, 1, 1, 12])
    model = objSARIMA.fit()
    print(model.summary().tables[0])
    print(model.summary().tables[1])
    modelData = {"model": model, "X_label": list(df.reset_index().columns)[0],
                 "y_label": list(df.reset_index().columns)[1]}
    return modelData


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run srimassHooks.ipynb
try:
	#sourcePreExecutionHook()

	sssalehitorykzga = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1716367374465_SSSalesHistory.csv', 'FilePath': '/TimeSeries/SuperStore/dataFiles/SSSalesHistory.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'SSSalesHistory.csv', 'viewFileName': 'SSSalesHistory.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run srimassHooks.ipynb
try:
	#transformationPreExecutionHook()

	timeseriesfe = TransformationTimeSeriesForecastingMain.run(sssalehitorykzga, json.dumps( {"FE": {"functionList": [{"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{}], "feature": "Order_Date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Order_Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sales", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1472.98", "stddev": "2147.85", "min": "3.928", "max": "28106.716", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Sales"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_First Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "1.55", "min": "-0.189470089", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_First Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Same Day", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "0.81", "min": "-0.007506949", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Same Day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Second Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.19", "stddev": "1.87", "min": "-0.661398746", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Second Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Ship Mode_Standard Class", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.86", "stddev": "3.39", "min": "-0.483581511", "max": "24.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Ship Mode_Standard Class"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Consumer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.28", "stddev": "3.25", "min": "-0.534658316", "max": "19.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Consumer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Corporate", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.92", "stddev": "2.24", "min": "-0.209610384", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Corporate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Segment_Home Office", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.01", "stddev": "1.66", "min": "-0.308929391", "max": "11.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Segment_Home Office"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_Central", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.44", "stddev": "2.13", "min": "-0.475643886", "max": "17.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_Central"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_East", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.67", "stddev": "2.26", "min": "-0.623750938", "max": "13.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_East"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_South", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.06", "stddev": "1.62", "min": "-0.094892784", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_South"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Region_West", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2.04", "stddev": "2.47", "min": "-0.395274925", "max": "12.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Region_West"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Furniture", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.36", "stddev": "1.43", "min": "-0.018243082", "max": "8.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Furniture"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Office Supplies", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.71", "stddev": "2.98", "min": "-0.339093471", "max": "17.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Office Supplies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Category_Technology", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1.14", "stddev": "1.29", "min": "-0.165872422", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Category_Technology"}], "features": {"timecolumn": "Order_Date", "tocompare": "Sales"}, "dataPercentage": 100, "originalfile": "/FileStore/platform/testdata/1716367374465_SSSalesHistory.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(timeseriesfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run srimassHooks.ipynb
try:
	#mlPreExecutionHook()

	modelData = driverSarima(timeseriesfe)

	#mlPostExecutionHook(modelData)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
from matplotlib import pyplot as plt
try:
    model=modelData['model']
    X_label=modelData['X_label']
    y_label=modelData['y_label']
    plt.clf()
    model.predict(1,264 )
    plt.title('Trending Graphs')
    plt.xlabel(X_label )
    plt.ylabel(y_label )
    plt.grid(color='lightgray')
    display()
except Exception as ex:
    logging.error(ex)

spark.stop()


